In [1]:
from anomaly_detection_helper import *

In [148]:
import imp
import anomaly_detection_helper
imp.reload(anomaly_detection_helper)
from anomaly_detection_helper import *

In [101]:
full_optiondata = pd.read_parquet('TSLA_2014_2021.parquet')
splitdate = '2020-08-31'
splitratio = 5
adj_full_optiondata = adjust_stock_split(full_optiondata,splitdate,splitratio)

# Train TSLA on 2014/1 -  2020/10, test on 2020/11 - 2021/5 

## A. Test window size 10/60/252

### A.1 window size 10

In [167]:
train_test_split_date = '2020-11-01'
train, test = train_test_split(adj_full_optiondata, train_test_split_date, window=10)

cur_month, next_month, third_month = monthly_contract(train)
atm, train_otm, itm = moneyness_contract(next_month)

cur_month, next_month, third_month = monthly_contract(test)
atm, test_otm, itm = moneyness_contract(next_month)

In [168]:
performance1 = optimize_threshold(train_otm, _type='call', variable=['volume','pcratio','impliedvol'], 
                                  direction=['>','<','>'], volume_operation='sum', window=10, win_thresh=-0.01)

Testing 0th combo
Testing 500th combo
Testing 1000th combo
Testing 1500th combo
Testing 2000th combo


In [169]:
filter_threshold(performance1, winprob=0.7, topn=1).iloc[:5]

,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
766,0.701,87,0.085,-0.042,0.047027,"[1.6, 1.5, 0.6]"
986,0.701,77,0.090,-0.042,0.050532,"[1.8, 1.5, 0.6]"
725,0.703,74,0.074,-0.041,0.039845,"[1.6, 1.1, 0.5]"
726,0.704,71,0.072,-0.042,0.038256,"[1.6, 1.1, 0.6]"
375,0.706,68,0.069,-0.038,0.037542,"[1.3, 0.9, 0.5]"


In [170]:
outlier1 = anomaly_dates(test_otm, _type='call', variable=['volume','pcratio','impliedvol'], direction=['>','<','>'], 
                         thresh=[1.6, 1.5, 0.6], volume_operation='sum', window=10)
outlier_impact(test_otm, outlier1, win_thresh=-0.01)

+1 day stock price increase probability: 0.375
+1 day stock price avg increase: 0.049
+1 day stock price avg decrease -0.024 

+5 day stock price increase probability: 0.375
+5 day stock price avg increase: 0.178
+5 day stock price avg decrease: -0.038
+5 day expected return: 0.043000000000000003


,outlierday,nextday,next5thday
quotedate,,,
2020-11-17,440.95,484.02,554.91
2021-01-25,872.90,886.99,837.45
2021-02-22,717.29,710.40,715.46
2021-02-23,710.40,736.74,688.60
2021-03-26,612.70,609.20,695.05
2021-04-21,740.97,721.25,697.63
2021-04-26,737.02,707.27,685.35
2021-05-12,593.91,574.00,580.55


### A.2 window size 60

In [24]:
train_test_split_date = '2020-11-01'
train, test = train_test_split(adj_full_optiondata, train_test_split_date, window=60)

cur_month, next_month, third_month = monthly_contract(train)
atm, train_otm, itm = moneyness_contract(next_month)

cur_month, next_month, third_month = monthly_contract(test)
atm, test_otm, itm = moneyness_contract(next_month)

In [28]:
performance2 = optimize_threshold(train_otm, _type='call', variable=['volume','pcratio','impliedvol'], 
                                  direction=['>','<','>'], volume_operation='sum', window=60, win_thresh=-0.01)

Testing 0th combo
Testing 500th combo
Testing 1000th combo
Testing 1500th combo
Testing 2000th combo


In [30]:
filter_threshold(performance2, winprob=0.7, topn=1).iloc[:5]

,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
103,0.705,149,0.072,-0.053,0.035125,"[1.0, 1.5, 0.3]"
320,0.713,143,0.070,-0.044,0.037282,"[1.2, 1.5, 0.0]"
321,0.725,138,0.071,-0.044,0.039375,"[1.2, 1.5, 0.1]"
430,0.719,135,0.068,-0.044,0.036528,"[1.3, 1.5, 0.0]"
310,0.709,134,0.069,-0.041,0.036990,"[1.2, 1.4, 0.0]"


In [110]:
outlier2 = anomaly_dates(test_otm, _type='call', variable=['volume','pcratio','impliedvol'], direction=['>','<','>'], 
                         thresh=[1.0, 1.5, 0.3], volume_operation='sum', window=60)
outlier_impact(test_otm, outlier2, win_thresh=-0.01)

+1 day stock price increase probability: 0.611
+1 day stock price avg increase: 0.037
+1 day stock price avg decrease -0.016 

+5 day stock price increase probability: 0.722
+5 day stock price avg increase: 0.086
+5 day stock price avg decrease: -0.059
+5 day expected return: 0.045689999999999995


,outlierday,nextday,next5thday
quotedate,,,
2020-11-17,440.95,484.02,554.91
2020-11-24,554.91,572.91,568.43
2020-11-25,572.91,585.15,593.64
2020-11-30,574.53,582.21,643.69
2020-12-09,606.33,624.33,622.51
2020-12-16,622.51,651.42,645.92
2020-12-21,651.15,638.46,664.66
2020-12-22,638.46,645.92,693.24
2020-12-28,666.01,664.66,732.77


### A.3 window size 252

In [32]:
train_test_split_date = '2020-11-01'
train, test = train_test_split(adj_full_optiondata, train_test_split_date, window=252)

cur_month, next_month, third_month = monthly_contract(train)
atm, train_otm, itm = moneyness_contract(next_month)

cur_month, next_month, third_month = monthly_contract(test)
atm, test_otm, itm = moneyness_contract(next_month)

In [34]:
performance3 = optimize_threshold(train_otm, _type='call', variable=['volume','pcratio','impliedvol'], 
                                  direction=['>','<','>'], volume_operation='sum', window=252, win_thresh=-0.01)

Testing 0th combo
Testing 500th combo
Testing 1000th combo
Testing 1500th combo
Testing 2000th combo


In [35]:
filter_threshold(performance3, winprob=0.7, topn=1).iloc[:5]

,winprob,outliernum,posret,negret,expret,volume/pcratio/impliedvol
210,0.705,139,0.069,-0.051,0.033600,"[1.1, 1.5, 0.0]"
211,0.711,135,0.070,-0.053,0.034453,"[1.1, 1.5, 0.1]"
320,0.705,129,0.067,-0.049,0.032780,"[1.2, 1.5, 0.0]"
212,0.711,128,0.069,-0.054,0.033453,"[1.1, 1.5, 0.2]"
321,0.712,125,0.067,-0.050,0.033304,"[1.2, 1.5, 0.1]"


In [36]:
outlier3 = anomaly_dates(test_otm, _type='call', variable=['volume','pcratio','impliedvol'], direction=['>','<','>'], 
                         thresh=[1.1, 1.5, 0.0], volume_operation='sum', window=252)
outlier_impact(test_otm, outlier3, win_thresh=-0.01)

+1 day stock price increase probability: 0.429
+1 day stock price avg increase: 0.041
+1 day stock price avg decrease -0.016 

+5 day stock price increase probability: 0.714
+5 day stock price avg increase: 0.097
+5 day stock price avg decrease: -0.019
+5 day expected return: 0.063824


,outlierday,nextday,next5thday
quotedate,,,
2020-12-31,704.80,729.92,875.55
2021-02-22,717.29,710.40,715.46
2021-02-23,710.40,736.74,688.60
2021-03-26,612.70,609.20,695.05
2021-03-31,668.49,659.92,682.17
2021-04-01,659.92,695.05,675.65
2021-05-12,593.91,574.00,580.55


## B. Test variable

In [199]:
train_test_split_date = '2020-11-01'
train, test = train_test_split(adj_full_optiondata, train_test_split_date, window=60)

cur_month, next_month, third_month = monthly_contract(train)
atm, train_otm, itm = moneyness_contract(next_month)

cur_month, next_month, third_month = monthly_contract(test)
atm, test_otm, itm = moneyness_contract(next_month)

### B1. replace volume with open interest

In [189]:
performance4 = optimize_threshold(train_otm, _type='call', variable=['openinterest','pcratio','impliedvol'], 
                                  direction=['>','<','>'], volume_operation='sum', window=60, win_thresh=-0.01)

Testing 0th combo
Testing 500th combo
Testing 1000th combo
Testing 1500th combo
Testing 2000th combo


In [190]:
filter_threshold(performance4, winprob=0.7, topn=1).iloc[:5]

,winprob,outliernum,posret,negret,expret,openinterest/pcratio/impliedvol
1480,0.717,53,0.060,-0.043,0.030851,"[2.3, 1.0, 0.0]"
1535,0.739,46,0.073,-0.037,0.044290,"[2.3, 1.5, 0.5]"
1525,0.721,43,0.076,-0.037,0.044473,"[2.3, 1.4, 0.5]"
1645,0.738,42,0.076,-0.035,0.046918,"[2.4, 1.5, 0.5]"
1515,0.707,41,0.078,-0.037,0.044305,"[2.3, 1.3, 0.5]"


In [191]:
outlier4 = anomaly_dates(test_otm, _type='call', variable=['openinterest','pcratio','impliedvol'], direction=['>','<','>'], 
                         thresh=[2.3, 1.0, 0.0], volume_operation='sum', window=60)
outlier_impact(test_otm, outlier4, win_thresh=-0.01)

+1 day stock price increase probability: 0.667
+1 day stock price avg increase: 0.023
+1 day stock price avg decrease -0.005 

+5 day stock price increase probability: 0.333
+5 day stock price avg increase: 0.082
+5 day stock price avg decrease: -0.077
+5 day expected return: -0.024052999999999998


,outlierday,nextday,next5thday
quotedate,,,
2020-12-22,638.46,645.92,693.24
2021-02-16,796.70,795.77,710.40
2021-02-22,717.29,710.40,715.46
2021-02-23,710.40,736.74,688.60
2021-02-25,685.18,685.35,616.96
2021-02-26,685.35,715.46,601.46


### B2. use only volume & impliedvol

In [192]:
performance5 = optimize_threshold(train_otm, _type='call', variable=['volume','impliedvol'], 
                                  direction=['>','>'], volume_operation='sum', window=60, win_thresh=-0.01)

Testing 0th combo


In [193]:
filter_threshold(performance5, winprob=0.7, topn=1).iloc[:5]

,winprob,outliernum,posret,negret,expret,volume/impliedvol
23,0.701,144,0.069,-0.050,0.033419,"[1.2, 0.3]"
5,0.702,141,0.071,-0.061,0.031664,"[1.0, 0.5]"
6,0.721,122,0.073,-0.054,0.037567,"[1.0, 0.6]"
53,0.706,119,0.070,-0.050,0.034720,"[1.5, 0.3]"
25,0.719,114,0.068,-0.056,0.033156,"[1.2, 0.5]"


In [203]:
outlier5 = anomaly_dates(test_otm, _type='call', variable=['volume','impliedvol'], direction=['>','>'], 
                         thresh=[1.2, 0.3], volume_operation='sum', window=60)
outlier_impact(test_otm, outlier5, win_thresh=-0.01)

+1 day stock price increase probability: 0.615
+1 day stock price avg increase: 0.032
+1 day stock price avg decrease -0.016 

+5 day stock price increase probability: 0.846
+5 day stock price avg increase: 0.091
+5 day stock price avg decrease: -0.019
+5 day expected return: 0.07406


,outlierday,nextday,next5thday
quotedate,,,
2020-11-17,440.95,484.02,554.91
2020-11-25,572.91,585.15,593.64
2020-11-30,574.53,582.21,643.69
2020-12-09,606.33,624.33,622.51
2020-12-18,638.61,651.15,666.01
2020-12-21,651.15,638.46,664.66
2020-12-22,638.46,645.92,693.24
2020-12-31,704.80,729.92,875.55
2021-02-22,717.29,710.40,715.46


### B3. log transform volume

In [200]:
performance6 = optimize_threshold(train_otm, _type='call', variable=['volume','impliedvol'], 
                                  direction=['>','>'], volume_operation='sum', window=60, win_thresh=-0.01, log_trans=True)

Testing 0th combo


In [201]:
filter_threshold(performance6, winprob=0.7, topn=1).iloc[:5]

,winprob,outliernum,posret,negret,expret,volume/impliedvol
5,0.720,161,0.071,-0.066,0.032640,"[1.0, 0.5]"
15,0.720,150,0.071,-0.065,0.032920,"[1.1, 0.5]"
6,0.741,139,0.074,-0.060,0.039294,"[1.0, 0.6]"
16,0.738,130,0.075,-0.060,0.039630,"[1.1, 0.6]"
43,0.701,127,0.067,-0.049,0.032316,"[1.4, 0.3]"


In [202]:
outlier6 = anomaly_dates(test_otm, _type='call', variable=['volume','impliedvol'], direction=['>','>'], 
                         thresh=[1.0, 0.5], volume_operation='sum', window=60, log_trans=True)
outlier_impact(test_otm, outlier6, win_thresh=-0.01)

+1 day stock price increase probability: 0.667
+1 day stock price avg increase: 0.033
+1 day stock price avg decrease -0.017 

+5 day stock price increase probability: 0.833
+5 day stock price avg increase: 0.069
+5 day stock price avg decrease: -0.077
+5 day expected return: 0.044618


,outlierday,nextday,next5thday
quotedate,,,
2020-11-19,498.13,490.29,585.15
2020-11-23,523.62,554.91,582.21
2020-11-24,554.91,572.91,568.43
2020-11-25,572.91,585.15,593.64
2020-11-30,574.53,582.21,643.69
2020-12-09,606.33,624.33,622.51
2020-12-16,622.51,651.42,645.92
2020-12-18,638.61,651.15,666.01
2021-02-26,685.35,715.46,601.46
